# 데이터 처리

공격 탐지를 위한 데이터에는 정수형 데이터만 존재하지 않는다.

따라서 데이터를 학습시키기 위해서는 정수형 데이터로의 변환이 필요하다.

우선 KDDTest+ 의 형식을 확인하자.

    data를 보면, header가 없기때문에 컬렴명을 생성해주도록 header을 None으로 설정하자.


In [1]:
import pandas as pd

#train = pd.read_csv("./data/KDDTrain+.csv")
train = pd.read_csv("./data/KDDTrain+.csv", header = None)
train

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


data는 위와 같은 형식이고, 모든 요소들이 정수로만 이루어져있지 않은것을 확인 할 수 있다.

문자열 데이터를 정수형으로 바꾸기 위해서 해야하는 작업을 알아보자.

    1. 원 핫 인코딩

위 데이터 자료에서 2번째 열에 해당하는 카테고리는 전송 방식이다.

전송 방식은 Tcp, udp와 같이 여러 방식이 존재한다.

이때 각 행은 하나의 시간대의 로그기록으로, 전송 방식의 종류가 다르다.

이때 원 핫 인코딩을 한다면, 새로운 컬럼(전송방식 - tcp , udp 등등)을 생성하여 

각 행에 해당 전송방식이라면 1, 아니면 0 으로 나누는 과정을 의미한다.

즉 원 핫 인코딩을 통해 (기존 행 갯수 * 해당 컬럼의 카테고리 갯수)의 1, 0으로 채워져 있는 새로운 행렬을 생성한다.

    2. 침입 탐지 구분

침입 탐지는 기본적으로 정상 진입 vs 침입 이다.

따라서 분류가 기본이므로, 정답에 대한 결과값을 0과 1로 분류해 주어야한다.

---

## 1.원 핫 인코딩

그럼 원 핫 인코딩 부터 진행해 보자.

원 핫 인코딩을 하기 위해서 pandas의 명령어를 이용한다.

    pd.get_dummies()

위에서 확인한대로 문자열 데이터는 1, 2, 3열이다.(첫번째 열이 0 부터 시작 주의)

따라서 우리는 1, 2, 3열의 데이터 프레임을 선택하여 인코딩을 진행한다.

    pandas에서 여러개의 열을 선택할 때는 리스트 안에 컬럼명을 리스트로 넣어야한다.

In [2]:
selected = train[[1, 2, 3]]
encoded = pd.get_dummies(selected)
encoded = encoded.astype(int)
#위처럼 astype을 설정해주지 않으면 True와 False로 분류된다.
encoded

,1_icmp,1_tcp,1_udp,2_IRC,2_X11,2_Z39_50,2_aol,2_auth,2_bgp,2_courier,...,3_REJ,3_RSTO,3_RSTOS0,3_RSTR,3_S0,3_S1,3_S2,3_S3,3_SF,3_SH
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
125969,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
125970,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
125971,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


## 2.정답 라벨링

정답 라벨링은 trian 데이터의 마지막에서 2번째열에 normal 컬럼 부분이다.

해당 부분을 추출하기 위해 _'iloc'_ 을 통해 시리즈 형태로 추출하자.

In [3]:
label = train.iloc[:, -2]
label

0          normal
1          normal
2         neptune
3          normal
4          normal
           ...   
125968    neptune
125969     normal
125970     normal
125971    neptune
125972     normal
Name: 41, Length: 125973, dtype: object

마찬가지로 라벨에 대해서도 정수형 데이터로 반환이 필요해 보인다.

이때 0과 1 로 정답을 분류해야하기 때문에 normal과 normal을 제외한 모든 카테고리를 분류해보자.

(1)pandas의 series.unique()를 통해 카테고리를 추출하자.

(2)pandas의 sorted()를 통해 추출한 카테고리를 리스트 형태로 반환하자.

In [4]:
unique_label = sorted(label.unique())
unique_label

['back',
 'buffer_overflow',
 'ftp_write',
 'guess_passwd',
 'imap',
 'ipsweep',
 'land',
 'loadmodule',
 'multihop',
 'neptune',
 'nmap',
 'normal',
 'perl',
 'phf',
 'pod',
 'portsweep',
 'rootkit',
 'satan',
 'smurf',
 'spy',
 'teardrop',
 'warezclient',
 'warezmaster']

리스트 컴프리핸션을 이용하여 각 라벨에 정수를 부여하자. 

    이때 1차원 리스트를 enumrate로 추출하면 idx를 뽑을 수 있다.

In [5]:
label_to_int = {label: idx for idx, label in enumerate(unique_label)}
label_to_int

{'back': 0,
 'buffer_overflow': 1,
 'ftp_write': 2,
 'guess_passwd': 3,
 'imap': 4,
 'ipsweep': 5,
 'land': 6,
 'loadmodule': 7,
 'multihop': 8,
 'neptune': 9,
 'nmap': 10,
 'normal': 11,
 'perl': 12,
 'phf': 13,
 'pod': 14,
 'portsweep': 15,
 'rootkit': 16,
 'satan': 17,
 'smurf': 18,
 'spy': 19,
 'teardrop': 20,
 'warezclient': 21,
 'warezmaster': 22}

그러나 우리는 normal을 제외한 모든 값을 1로 만들것이므로 다른 방법을 시도하자.

pandas의 map 함수는 series의 해당 값을 바꿔주는데 사용한다.

즉 label 시리즈의 변수 x를 'normal'이면 0 그외에는 1

In [7]:
y = label.map(lambda x: 0 if x == 'normal' else 1)
y

0         0
1         0
2         1
3         0
4         0
         ..
125968    1
125969    0
125970    0
125971    1
125972    0
Name: 41, Length: 125973, dtype: int64

문자열 데이터는 이와 같은 방식으로 pandas 라이브러리를 이용하여 처리하면 된다.